In [1]:
pip install backtrader

     |████████████████████████████████| 410 kB 5.3 MB/s 


In [5]:
import backtrader as bt
import pandas as pd
import datetime
import matplotlib
%matplotlib inline


SIMPLE MOVING AVERAGE CROSSOVER


In [6]:
class MAcrossover(bt.Strategy): 
    # Moving average parameters
    params = (('pfast',7),('pslow',90),)

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} {txt}') # Comment this line when running optimization

    def __init__(self):
        self.dataclose = self.datas[0].close
        
		# Order variable will contain ongoing order details/status
        self.order = None
        self.bought = None
        self.sold = None

        # Instantiate moving averages
        self.slow_sma = bt.indicators.MovingAverageSimple(self.datas[0], 
                        period=self.params.pslow)
        self.fast_sma = bt.indicators.MovingAverageSimple(self.datas[0], 
                        period=self.params.pfast)
        self.crossover = bt.indicators.CrossOver(self.fast_sma, self.slow_sma)
        
    def notify_order(self, order):
      if order.status in [order.Submitted, order.Accepted]:
        # An active Buy/Sell order has been submitted/accepted - Nothing to do
        return

      # Check if an order has been completed
      # Attention: broker could reject order if not enough cash
      if order.status in [order.Completed]:
        if order.isbuy():
          self.log(f'BUY EXECUTED, {order.executed.price:.2f}')
        elif order.issell():
          self.log(f'SELL EXECUTED, {order.executed.price:.2f}')
        self.bar_executed = len(self)

      elif order.status in [order.Canceled, order.Margin, order.Rejected]:
        self.log('Order Canceled/Margin/Rejected')

    # Reset orders
      self.order = None


    def next(self):
      # Check for open orders
      if self.order:
        return

      # Check if we are in the market
      if not self.position:
        # We are not in the market, look for a signal to OPEN trades
          
        #If the 20 SMA is above the 50 SMA
        if self.crossover > 0:
          self.log(f'BUY CREATE {self.dataclose[0]:2f}')
          # Keep track of the created order to avoid a 2nd order
          self.order = self.buy()
          self.bought = True
        #Otherwise if the 20 SMA is below the 50 SMA   
        elif self.crossover < 0:
          self.log(f'SELL CREATE {self.dataclose[0]:2f}')
          # Keep track of the created order to avoid a 2nd order
          self.order = self.sell()
          self.sold = True
      else:
        # We are already in the market, look for a signal to CLOSE trades
        if self.bought:
          if self.crossover < 0:
            self.log(f'CLOSE CREATE {self.dataclose[0]:2f}')
            self.order = self.close()  
            self.bought = None
        if self.sold:
          if self.crossover > 0:
            self.log(f'CLOSE CREATE {self.dataclose[0]:2f}')
            self.order = self.close()  
            self.sold = None
        
          
            

In [14]:
#Instantiate Cerebro engine
cerebro = bt.Cerebro()

#Add strategy to Cerebro
cerebro.addstrategy(MAcrossover)
data = bt.feeds.YahooFinanceCSVData(dataname='/content/INFY_NS.csv',
    fromdate=datetime.datetime(2000, 12, 25),
    todate=datetime.datetime(2020, 5, 5))

cerebro.adddata(data)

    # Set our desired cash start

cerebro.broker.setcash(100000.0)

cerebro.addsizer(bt.sizers.SizerFix, stake=70)
    # Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
cerebro.run()
cerebro.plot()

    # Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2001-08-06 BUY CREATE 41.690000
2001-08-07 BUY EXECUTED, 41.25
2001-08-16 CLOSE CREATE 41.290000
2001-08-17 SELL EXECUTED, 41.26
2001-08-27 BUY CREATE 43.060000
2001-08-28 BUY EXECUTED, 42.32
2001-09-03 CLOSE CREATE 37.640000
2001-09-04 SELL EXECUTED, 37.36
2001-11-20 BUY CREATE 39.640000
2001-11-21 BUY EXECUTED, 39.14
2002-02-22 CLOSE CREATE 40.750000
2002-02-25 SELL EXECUTED, 40.76
2002-03-08 BUY CREATE 46.620000
2002-03-11 BUY EXECUTED, 46.56
2002-03-22 CLOSE CREATE 42.240000
2002-03-25 SELL EXECUTED, 42.24
2002-04-24 BUY CREATE 42.800000
2002-04-25 BUY EXECUTED, 42.64
2002-04-26 CLOSE CREATE 41.940000
2002-04-29 SELL EXECUTED, 41.77
2002-05-16 BUY CREATE 41.980000
2002-05-17 BUY EXECUTED, 41.99
2002-05-21 CLOSE CREATE 39.950000
2002-05-22 SELL EXECUTED, 39.58
2002-08-28 BUY CREATE 37.890000
2002-08-29 BUY EXECUTED, 37.64
2003-02-13 CLOSE CREATE 46.310000
2003-02-14 SELL EXECUTED, 46.37
2003-07-11 BUY CREATE 39.210000
2003-07-14 BUY EXECUTED, 39.8

BB crossover (single stock)

In [37]:
class BB2CrossoverSingle(bt.Strategy): 
    # Moving average parameters
    params = (('period', 20),('devfactor1', 1),('devfactor2', 2),)
    

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} {txt}') # Comment this line when running optimization

    def __init__(self):
        
        self.dataclose = self.datas[0].close
        self.dataopen = self.datas[0].open
        
		# Order variable will contain ongoing order details/status
        self.order = None
        self.bought = None
        self.sold = None

        # Instantiate the bollinger bands
        self.bband_1 = bt.indicators.BBands(self.datas[0], period=self.params.period, devfactor = self.params.devfactor1)
        self.bband_2 = bt.indicators.BBands(self.datas[0], period=self.params.period, devfactor = self.params.devfactor2)
        
        self.crossoversell = bt.indicators.CrossOver(self.dataclose, self.bband_2.lines.top , plot = False) #if it closes in the second band
        
        
        self.crossoverbuy = bt.indicators.CrossOver(self.dataclose,self.bband_1.lines.top,  plot = False) #if it closes in the second band
        
        

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
          # An active Buy/Sell order has been submitted/accepted - Nothing to do
          return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
          if order.isbuy():
            self.log(f'BUY EXECUTED, {order.executed.price:.2f}')
          elif order.issell():
            self.log(f'SELL EXECUTED, {order.executed.price:.2f}')
          self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
          self.log('Order Canceled/Margin/Rejected')

        # Reset orders
        self.order = None 


    def next(self):
      # Check for open orders
      if self.order:
        return

      # Check if we are in the market
      if not self.position:
        # We are not in the market, look for a signal to OPEN trades
          
        if self.crossoverbuy > 0: # buy signal
          self.log(f'BUY CREATE {self.dataclose[0]:2f}')
          # Keep track of the created order to avoid a 2nd order
          self.order = self.buy()
          self.bought = True
          # self.sold = False


        # elif self.crossoversell > 0: # Fast ma crosses below slow ma
        #   self.log(f'SELL CREATE {self.dataclose[0]:2f}')
        #   # Keep track of the created order to avoid a 2nd order
        #   self.order = self.sell()
        #   self.sold = True
        #   # self.bought = False

        
      else:
        # We are already in the market, look for a signal to CLOSE trades
        if self.bought:
          if self.crossoverbuy < 0 and self.dataopen[0] > self.dataclose[0] : # exit signal
            self.log(f'CLOSE CREATE BUY {self.dataclose[0]:2f}')
            self.order = self.close()
            self.bought = None

        # if self.sold:
        #   if self.crossoverbuy > 0:
        #     self.log(f'CLOSE CREATE SELL {self.dataclose[0]:2f}')
        #     self.order = self.close()
        #     self.sold = None

In [45]:
#Instantiate Cerebro engine
cerebro = bt.Cerebro()

#Add strategy to Cerebro
cerebro.addstrategy(BB2CrossoverSingle)
data = bt.feeds.YahooFinanceCSVData(dataname='/content/INFY_NS.csv',
    fromdate=datetime.datetime(2000, 12, 25),
    todate=datetime.datetime(2021, 5, 5))

cerebro.adddata(data)

    # Set our desired cash start

cerebro.broker.setcash(100000.0)

cerebro.addsizer(bt.sizers.SizerFix, stake=80)
    # Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
cerebro.run()
cerebro.plot()

    # Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2001-01-30 BUY CREATE 75.570000
2001-01-31 BUY EXECUTED, 75.17
2001-01-31 CLOSE CREATE BUY 74.520000
2001-02-01 SELL EXECUTED, 74.17
2001-05-07 BUY CREATE 43.900000
2001-05-08 BUY EXECUTED, 43.70
2001-05-09 CLOSE CREATE BUY 42.470000
2001-05-10 SELL EXECUTED, 42.24
2001-05-23 BUY CREATE 46.420000
2001-05-24 BUY EXECUTED, 46.17
2001-05-30 CLOSE CREATE BUY 43.190000
2001-05-31 SELL EXECUTED, 41.22
2001-07-11 BUY CREATE 41.180000
2001-07-12 BUY EXECUTED, 41.94
2001-07-13 CLOSE CREATE BUY 40.600000
2001-07-16 SELL EXECUTED, 40.79
2001-07-20 BUY CREATE 41.350000
2001-07-23 BUY EXECUTED, 40.65
2001-07-23 CLOSE CREATE BUY 39.820000
2001-07-24 SELL EXECUTED, 39.63
2001-08-03 BUY CREATE 42.480000
2001-08-06 BUY EXECUTED, 42.49
2001-08-29 CLOSE CREATE BUY 41.290000
2001-08-30 SELL EXECUTED, 41.66
2001-10-11 BUY CREATE 30.330000
2001-10-12 BUY EXECUTED, 31.88
2001-12-13 CLOSE CREATE BUY 47.290000
2001-12-14 SELL EXECUTED, 45.47
2002-01-04 BUY CREATE 49.420000
2

BB Crossover - Multiple 

In [54]:
class BB2CrossoverMUL(bt.Strategy): 
    # Moving average parameters
    params = (('period', 20),('devfactor1', 1),('devfactor2', 2),)

    def __init__(self):
        self.inds = dict()
        for i, d in enumerate(self.datas):
            self.inds[d] = dict()
            self.inds[d]['bbands'] = bt.indicators.BBands(d.close, period=self.params.period, devfactor = self.params.devfactor1).top
            self.inds[d]['cross'] = bt.indicators.CrossOver(d.close,self.inds[d]['bbands'],  plot = False) 
            print(self.inds[d])
          
    def next(self):
        for i, d in enumerate(self.datas):
          dt, dn = self.datetime.date(), d._name
          pos = self.getposition(d).size
          if not pos:  # no market / no orders
            if self.inds[d]['cross'][0] > 0:
              self.buy(data=d, size = 50)
          else:
            if self.inds[d]['cross'][0] < 0 and d.close < d.open:
              self.close(data=d)
              # self.sell(data=d, size=10)

    def notify_trade(self, trade):
      dt = self.data.datetime.date()
      if trade.isclosed:
        print('{} {} Closed: PnL Gross {}, Net {}'.format(
                                            dt,
                                            trade.data._name,
                                            round(trade.pnl,2),
                                            round(trade.pnlcomm,2)))
                    



In [59]:
#Instantiate Cerebro engine
cerebro = bt.Cerebro()

#Add strategy to Cerebro
cerebro.addstrategy(BB2CrossoverMUL)
datalist = [
    ('/content/INFY_NS.csv', 'INFY'),
    ('/content/ASIANPAINT_NS.csv', 'ASIANPAINT'),
    ('/content/BAJFINANCE_NS.csv', 'BAJAJ'),
    ('/content/BHARTIARTL_NS.csv', 'BHARTI'),
    ('/content/HDFCBAN_NS.csv', 'HDFC'),
    ('/content/HINDUNILVR_NS.csv', 'HUL'),
    ('/content/ICICIBANK_NS.csv', 'ICICI'),
    ('/content/KOTAKBANK_NS.csv', 'KOTAK'),
    ('/content/RELIANCE_NS.csv', 'RELIANCE'),
    ('/content/TCS_NS.csv', 'TCS'),
    ('/content/SBIN_NS.csv', 'SBI'),
    ('/content/WIPRO_NS.csv', 'WIPRO'),

]

for i in range(len(datalist)):
    data = bt.feeds.YahooFinanceCSVData(dataname=datalist[i][0],
    fromdate=datetime.datetime(2000, 12, 25),
    todate=datetime.datetime(2021, 5, 5))
    cerebro.adddata(data, name=datalist[i][1])



    # Set our desired cash start

cerebro.broker.setcash(100000.0)

# cerebro.addsizer(bt.sizers.SizerFix, stake=40)
    # Print out the starting conditions
initial = cerebro.broker.getvalue()
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
cerebro.run()
# cerebro.plot()

    # Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
final = cerebro.broker.getvalue()
print('Final Portfolio Percentage: %.2f' % ((final - initial)/initial*100))

Starting Portfolio Value: 100000.00
{'bbands': <backtrader.linebuffer.LineBuffer object at 0x7f1fbba82c90>, 'cross': <backtrader.indicators.crossover.CrossOver object at 0x7f1fbba725d0>}
{'bbands': <backtrader.linebuffer.LineBuffer object at 0x7f1fbba70f50>, 'cross': <backtrader.indicators.crossover.CrossOver object at 0x7f1fbba5fad0>}
{'bbands': <backtrader.linebuffer.LineBuffer object at 0x7f1fb845d450>, 'cross': <backtrader.indicators.crossover.CrossOver object at 0x7f1fb844c050>}
{'bbands': <backtrader.linebuffer.LineBuffer object at 0x7f1fb844ea50>, 'cross': <backtrader.indicators.crossover.CrossOver object at 0x7f1fb84435d0>}
{'bbands': <backtrader.linebuffer.LineBuffer object at 0x7f1fb8446fd0>, 'cross': <backtrader.indicators.crossover.CrossOver object at 0x7f1fb845bad0>}
{'bbands': <backtrader.linebuffer.LineBuffer object at 0x7f1fb8487550>, 'cross': <backtrader.indicators.crossover.CrossOver object at 0x7f1fb84a3110>}
{'bbands': <backtrader.linebuffer.LineBuffer object at 0x7

In [11]:
cerebro = bt.Cerebro(optreturn=False)

#Set data parameters and add to Cerebro

data = bt.feeds.YahooFinanceCSVData(dataname='/content/INFY.csv',
    fromdate=datetime.datetime(2017, 1, 1),
    todate=datetime.datetime(2019, 12, 25))

cerebro.adddata(data)

#Add strategy to Cerebro
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio')
cerebro.optstrategy(MAcrossover, pfast=range(5, 20), pslow=range(50, 100))  

#Default position size
cerebro.addsizer(bt.sizers.SizerFix, stake=3)

if __name__ == '__main__':
    optimized_runs = cerebro.run()

    final_results_list = []
    for run in optimized_runs:
        for strategy in run:
            PnL = round(strategy.broker.get_value() - 10000,2)
            sharpe = strategy.analyzers.sharpe_ratio.get_analysis()
            final_results_list.append([strategy.params.pfast, 
                strategy.params.pslow, PnL, sharpe['sharperatio']])

    sort_by_sharpe = sorted(final_results_list, key=lambda x: x[3], 
                             reverse=True)
    for line in sort_by_sharpe[:5]:
        print(line)

[5, 92, 6.06, -24.06202735171744]
[5, 91, 6.33, -24.332972281000497]
[5, 90, 6.42, -24.48465207166556]
[5, 89, 6.87, -25.245520416955358]
[6, 89, 7.35, -25.66505286438338]


In [3]:
class TestStrategy(bt.Strategy):
    params = (('BBandsperiod', 20),)

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.redline = None
        self.blueline = None

        # Add a BBand indicator
        self.bband = bt.indicators.BBands(self.datas[0], period=self.params.BBandsperiod)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enougth cash
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        #if it closes below bottom, red line = true
        if self.dataclose < self.bband.lines.bot and not self.position:
        	self.redline = True


        if self.dataclose > self.bband.lines.top and self.position:
        	self.blueline = True

        # buy if it closes below mid line and it earlier closed below bottom
        if self.dataclose > self.bband.lines.mid and not self.position and self.redline:        	
        	# BUY, BUY, BUY!!! (with all possible default parameters)
            self.log('BUY CREATE, %.2f' % self.dataclose[0])
            # Keep track of the created order to avoid a 2nd order
            self.order = self.buy()

        #if it closed above top line, buy -> indicating uptrend?
        if self.dataclose > self.bband.lines.top and not self.position:
            # BUY, BUY, BUY!!! (with all possible default parameters)
            self.log('BUY CREATE, %.2f' % self.dataclose[0])
            # Keep track of the created order to avoid a 2nd order
            self.order = self.buy()

        if self.dataclose < self.bband.lines.mid and self.position and self.blueline:
            # SELL, SELL, SELL!!! (with all possible default parameters)
            self.log('SELL CREATE, %.2f' % self.dataclose[0])
            self.blueline = False
            self.redline = False
            # Keep track of the created order to avoid a 2nd order
            self.order = self.sell()

In [6]:
#Instantiate Cerebro engine
cerebro = bt.Cerebro()

#Add strategy to Cerebro
cerebro.addstrategy(TestStrategy)
data = bt.feeds.YahooFinanceCSVData(dataname='/content/INFY_NS.csv',
    fromdate=datetime.datetime(2001, 12, 25),
    todate=datetime.datetime(2020, 5, 5))

cerebro.adddata(data)

    # Set our desired cash start

cerebro.broker.setcash(100000.0)

cerebro.addsizer(bt.sizers.SizerFix, stake=40)
    # Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
cerebro.run()


    # Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2002-01-21, Close, 43.12
2002-01-22, Close, 44.04
2002-01-23, Close, 42.78
2002-01-24, Close, 42.53
2002-01-25, Close, 42.18
2002-01-28, Close, 41.80
2002-01-29, Close, 41.60
2002-01-30, Close, 40.42
2002-01-31, Close, 42.73
2002-02-01, Close, 42.14
2002-02-04, Close, 40.88
2002-02-05, Close, 41.28
2002-02-06, Close, 43.19
2002-02-07, Close, 40.48
2002-02-08, Close, 40.73
2002-02-11, Close, 41.28
2002-02-12, Close, 41.38
2002-02-13, Close, 41.75
2002-02-14, Close, 42.25
2002-02-15, Close, 42.28
2002-02-18, Close, 41.10
2002-02-19, Close, 40.85
2002-02-20, Close, 40.22
2002-02-21, Close, 40.31
2002-02-22, Close, 40.75
2002-02-25, Close, 41.28
2002-02-26, Close, 42.29
2002-02-27, Close, 42.26
2002-02-28, Close, 38.94
2002-03-01, Close, 39.83
2002-03-04, Close, 40.17
2002-03-05, Close, 42.23
2002-03-05, BUY CREATE, 42.23
2002-03-06, BUY EXECUTED, Price: 42.44, Cost: 1697.60, Comm 0.00
2002-03-06, Close, 43.22
2002-03-07, Close, 45.47
2002-03-08, Close, 

In [12]:
cerebro.plot()

[[<Figure size 432x288 with 4 Axes>]]